<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1SnLSbiQxs5WBhkR5LwXm5F-Lsy3SEsid?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- Join Innovation Community

Learn by building. Get expert mentorship and work on real AI projects.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)

# 🚀 Claude Sonnet 4.6 — Complete Testing Notebook

**Anthropic's Most Capable Sonnet Model Yet**

Claude Sonnet 4.6 is a full upgrade across coding, computer use, long-context reasoning, agent planning, knowledge work, and design. Developers with early access preferred Sonnet 4.6 over Sonnet 4.5 **~70% of the time**, and even preferred it to Opus 4.5 **59% of the time**.

---

## 📊 Model Specifications

| Feature | Specification |
|---------|---------------|
| **Model ID** | `anthropic/claude-sonnet-4.6` |
| **Context Window** | 1M tokens (beta) |
| **Max Output** | 128K tokens |
| **Modalities** | Text, Vision/Images |
| **Tool Calling** | ✅ Parallel & chained |
| **Adaptive Thinking** | ✅ Reasoning control |
| **Streaming** | ✅ Supported |
| **JSON Mode** | ✅ Structured output |
| **Input Pricing** | $3 / 1M tokens |
| **Output Pricing** | $15 / 1M tokens |

---

## 🔑 Key Capabilities (from Anthropic)

1. **Coding Excellence** — Preferred 70% over Sonnet 4.5 in Claude Code; reads context before modifying, consolidates shared logic
2. **Computer Use** — Major improvements on OSWorld benchmark; human-level at spreadsheets & multi-step web forms
3. **1M Token Context** — Hold entire codebases, contracts, or dozens of research papers in a single request
4. **Agent Planning** — Long-horizon planning with strategic pivoting (top performer on Vending-Bench Arena)
5. **Adaptive Thinking** — Adjusts reasoning depth; strong with or without extended thinking
6. **Tool Use** — Parallel calls, web search, code execution, memory, and programmatic tool calling
7. **Design & Frontend** — Polished layouts, animations, and design sensibility
8. **Instruction Following** — Fewer hallucinations, fewer false success claims, better follow-through on multi-step tasks

📖 *Source: [Anthropic — Introducing Claude Sonnet 4.6](https://www.anthropic.com/news/claude-sonnet-4-6)*

---
## 📦 Setup & Installation

In [1]:
# @title Install Dependencies
!pip install -q openai httpx

In [ ]:
# @title Configure API Client (OpenRouter)
from google.colab import userdata
from openai import OpenAI

# Setup OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY")
)

MODEL = "anthropic/claude-sonnet-4.6"

print(f"✅ Client configured for: {MODEL}")
print(f"🔗 Model page: https://openrouter.ai/anthropic/claude-sonnet-4.6")

---
## 🧪 Example 1: Basic Chat & Self-Introduction

Let's start by testing Sonnet 4.6's basic chat and see how it describes its own capabilities.

In [4]:
# @title Basic Chat Completion
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are Claude Sonnet 4.6 by Anthropic, accessed via OpenRouter. Be helpful and concise."},
        {"role": "user", "content": "What are your key strengths compared to previous Sonnet models? Be specific and concise."}
    ],
    max_tokens=1024
)

print("💬 Claude Sonnet 4.6 Response:")
print("=" * 50)
print(response.choices[0].message.content)

💬 Claude Sonnet 4.6 Response:
I'm Claude Sonnet 4.6, and compared to previous Sonnet models, my key improvements include:

1. **Enhanced reasoning** – Better at complex, multi-step problems and logical analysis
2. **Improved instruction-following** – More precise adherence to nuanced or detailed prompts
3. **Better coding** – More accurate code generation, debugging, and explanation
4. **Stronger factual accuracy** – Reduced hallucinations on knowledge-based tasks
5. **More nuanced writing** – Better tone calibration and stylistic control
6. **Improved agentic performance** – Better at tool use and multi-turn task completion

**Honest caveat:** I don't have detailed internal benchmarks comparing myself to specific prior Sonnet versions, so I can't give you precise, quantified differences. For rigorous comparisons, Anthropic's release notes and third-party benchmarks (like MMLU, HumanEval, etc.) are more reliable sources.


---
## 🛠️ Example 2: Advanced Tool Calling (Parallel Execution)

Sonnet 4.6 excels at tool use with parallel calls, web search, and programmatic tool calling. Let's test with multiple tools simultaneously.

In [5]:
# @title Define Tools
import json
from datetime import datetime

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "City name (e.g., 'Tokyo', 'New York')"},
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"], "description": "Temperature unit"}
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform a mathematical calculation",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "Math expression to evaluate (e.g., '15 * 0.18')"}
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Get the current stock price for a ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {"type": "string", "description": "Stock ticker symbol (e.g., 'AAPL', 'GOOGL')"}
                },
                "required": ["ticker"]
            }
        }
    }
]

# Simulate tool execution
def execute_tool(name, args):
    if name == "get_weather":
        location = args.get('location', 'Unknown')
        unit = args.get('unit', 'celsius')
        temp = 24 if unit == 'celsius' else 75
        return json.dumps({"location": location, "temperature": temp, "unit": unit, "condition": "Partly Cloudy", "humidity": 62})
    elif name == "calculate":
        try:
            result = eval(args.get('expression', '0'))
            return json.dumps({"result": result, "expression": args.get('expression')})
        except Exception as e:
            return json.dumps({"error": str(e)})
    elif name == "get_stock_price":
        prices = {"AAPL": 198.50, "GOOGL": 175.30, "MSFT": 425.80, "AMZN": 192.60}
        ticker = args.get('ticker', 'AAPL').upper()
        return json.dumps({"ticker": ticker, "price": prices.get(ticker, 150.00), "currency": "USD", "change": "+1.2%"})
    return json.dumps({"error": "Unknown tool"})

print("✅ Tools defined: get_weather, calculate, get_stock_price")

✅ Tools defined: get_weather, calculate, get_stock_price


In [6]:
# @title Parallel Tool Calling Demo
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "I need three things: 1) Weather in Tokyo, 2) Calculate 18% tip on $125.50, and 3) Current price of AAPL stock."}
    ],
    tools=tools,
    max_tokens=1024
)

message = response.choices[0].message
print("📤 Model Response:")

if message.tool_calls:
    print(f"\n🔧 Tool Calls Detected: {len(message.tool_calls)}")
    print(f"{'─' * 50}")

    # Process each tool call
    tool_messages = [{"role": "assistant", "content": message.content, "tool_calls": [{"id": tc.id, "type": "function", "function": {"name": tc.function.name, "arguments": tc.function.arguments}} for tc in message.tool_calls]}]

    for tc in message.tool_calls:
        args = json.loads(tc.function.arguments)
        result = execute_tool(tc.function.name, args)
        print(f"   ➤ {tc.function.name}({json.dumps(args)})")
        print(f"   📋 Result: {result}")
        print()
        tool_messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})

    # Get final response with tool results
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": "I need three things: 1) Weather in Tokyo, 2) Calculate 18% tip on $125.50, and 3) Current price of AAPL stock."},
        ] + tool_messages,
        tools=tools,
        max_tokens=1024
    )
    print(f"{'─' * 50}")
    print("\n✅ Final Answer:")
    print(final_response.choices[0].message.content)
else:
    print(message.content)

📤 Model Response:

🔧 Tool Calls Detected: 3
──────────────────────────────────────────────────
   ➤ get_weather({"location": "Tokyo"})
   📋 Result: {"location": "Tokyo", "temperature": 24, "unit": "celsius", "condition": "Partly Cloudy", "humidity": 62}

   ➤ calculate({"expression": "125.50 * 0.18"})
   📋 Result: {"result": 22.59, "expression": "125.50 * 0.18"}

   ➤ get_stock_price({"ticker": "AAPL"})
   📋 Result: {"ticker": "AAPL", "price": 198.5, "currency": "USD", "change": "+1.2%"}

──────────────────────────────────────────────────

✅ Final Answer:
Here are all three results:

1. 🌤️ **Weather in Tokyo**
   - Temperature: **24°C**
   - Condition: **Partly Cloudy**
   - Humidity: **62%**

2. 🧾 **18% Tip on $125.50**
   - Tip Amount: **$22.59**
   - Total (with tip): **$148.09**

3. 📈 **AAPL Stock Price**
   - Current Price: **$198.50 USD**
   - Change: **+1.2%** ▲

Let me know if you need anything else!


---
## 💻 Example 3: Code Generation & Debugging

Sonnet 4.6 brings much-improved coding skills — it reads context before modifying code, consolidates shared logic, and is less prone to overengineering. Users rated it significantly better at instruction following with fewer hallucinations.

In [ ]:
# @title Advanced Code Generation
coding_prompt = """
Create a Python `AsyncRateLimiter` class with the following requirements:

1. Token bucket algorithm with configurable rate and burst size
2. Async/await compatible (use asyncio)
3. Decorator pattern for easy function wrapping
4. Usage tracking with stats (total requests, throttled count, avg wait time)
5. Graceful shutdown support
6. Type hints and docstrings throughout

Include a demo showing it in action with 3 simulated API calls.
"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are an expert Python developer. Write clean, production-quality code. Include complete, runnable implementations."},
        {"role": "user", "content": coding_prompt}
    ],
    max_tokens=4096
)

print(response.choices[0].message.content)

In [ ]:
# @title Code Debugging & Refactoring
debug_prompt = """
Here's a buggy Python function. Find ALL bugs, explain each one, and provide the fixed version:

```python
def merge_sorted_lists(list1, list2):
    result = []
    i = j = 0

    while i <= len(list1) and j <= len(list2):
        if list1[i] < list2[j]:
            result.append(list1[i])
            i += 1
        else:
            result.append(list2[j])
            j += 1

    result += list1[i:]
    result += list2[j:]
    return result

def find_duplicates(lst):
    seen = {}
    duplicates = []
    for item in lst:
        if item in seen:
            duplicates.append(item)
        seen[item] = True
    return list(duplicates)
```

Also suggest performance improvements and add edge case handling.
"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are an expert code reviewer. Be thorough — identify ALL bugs, explain them clearly, and provide clean fixes."},
        {"role": "user", "content": debug_prompt}
    ],
    max_tokens=3000
)

print(response.choices[0].message.content)

---
## 📊 Example 4: Structured JSON Output

Sonnet 4.6 provides reliable structured output for data extraction, API responses, and pipeline integrations.

In [ ]:
# @title JSON Mode — Data Extraction
import json

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": """You are a data extraction specialist. Always respond with valid JSON matching this exact schema:
{
  "company": {"name": string, "industry": string, "founded": number, "hq": string},
  "products": [{"name": string, "category": string, "description": string}],
  "key_metrics": {"employees": string, "valuation": string, "funding": string},
  "competitors": [string],
  "notable_achievements": [string]
}"""},
        {"role": "user", "content": "Extract structured information about Anthropic, the AI safety company that created Claude."}
    ],
    max_tokens=1024,
    response_format={"type": "json_object"}
)

content = response.choices[0].message.content

try:
    result = json.loads(content)
    print("📋 Extracted Data:")
    print(json.dumps(result, indent=2))
except json.JSONDecodeError as e:
    print(f"❌ JSONDecodeError: {e}")
    print("Raw content received from model:")
    print("-" * 20)
    print(content if content else "[Empty Response]")
    print("-" * 20)

In [ ]:
# @title Complex Schema — Multi-Entity Extraction
text = """
In Q4 2025, OpenAI launched GPT-5.2 with multimodal reasoning. Google followed with Gemini 3 Pro,
which set new records on MMLU-Pro and GPQA benchmarks. Anthropic's Claude Opus 4.5 introduced
adaptive thinking and a 1M token context window. Meta released Llama 4.1, an open-weight model
that matched GPT-4-level performance. Meanwhile, Mistral launched their Pixtral Large model with
improved vision capabilities.
"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": """Extract all AI models mentioned. Return JSON array:
[{"model": string, "company": string, "key_features": [string], "release_period": string}]"""},
        {"role": "user", "content": f"Extract model information from this text:\n{text}"}
    ],
    max_tokens=1024,
    response_format={"type": "json_object"}
)

content = response.choices[0].message.content

try:
    # Strip markdown code blocks if present
    clean_content = content.strip()
    if clean_content.startswith("```"):
        # Find the end of the first line (e.g., ```json)
        first_newline = clean_content.find('\n')
        if first_newline != -1:
            clean_content = clean_content[first_newline:].strip()
    if clean_content.endswith("```"):
        clean_content = clean_content[:-3].strip()

    models = json.loads(clean_content)
    print("🤖 Extracted AI Models:")
    print(json.dumps(models, indent=2))
except json.JSONDecodeError as e:
    print(f"❌ JSONDecodeError: {e}")
    print("Raw content received from model:")
    print("-" * 20)
    print(content)
    print("-" * 20)

---
## 🌊 Example 5: Streaming Responses

Stream long-form content generation for real-time output.

In [ ]:
# @title Streaming with Token Counting
import time

stream = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Explain the key differences between Claude Sonnet 4.6 and GPT-5 from an AI architecture perspective. Be concise (3 paragraphs)."}
    ],
    max_tokens=1024,
    stream=True
)

full_response = ""
token_count = 0
start_time = time.time()

print("📡 Streaming response:\n")
print("─" * 50)

for chunk in stream:
    if chunk.choices[0].delta.content:
        content = chunk.choices[0].delta.content
        full_response += content
        token_count += 1
        print(content, end="", flush=True)

elapsed = time.time() - start_time
print(f"\n{'─' * 50}")
print(f"\n✅ Stream complete!")
print(f"⏱️ Time: {elapsed:.2f}s | 📊 Chunks: {token_count} | ⚡ Speed: {token_count/elapsed:.1f} chunks/s")

---
## 🔄 Example 6: Multi-Turn Conversation with Memory

Test Sonnet 4.6's ability to maintain context across multiple turns — a key capability for agentic workflows.

In [ ]:
# @title Multi-Turn Conversation
conversation = [
    {"role": "system", "content": "You are a helpful AI tutor specializing in machine learning. Keep responses concise and educational. Remember all context from the conversation."}
]

questions = [
    "What is a transformer architecture? Explain in 2-3 sentences.",
    "How does self-attention work in transformers? Use the concept you just explained.",
    "Now explain how this relates to modern LLMs like yourself. Connect it to your earlier explanations."
]

for i, question in enumerate(questions, 1):
    print(f"\n{'='*60}")
    print(f"👤 Turn {i}: {question}")
    print(f"{'='*60}")

    conversation.append({"role": "user", "content": question})

    response = client.chat.completions.create(
        model=MODEL,
        messages=conversation,
        max_tokens=512
    )

    reply = response.choices[0].message.content
    conversation.append({"role": "assistant", "content": reply})

    print(f"\n🤖 Claude: {reply}")

print(f"\n{'='*60}")
print(f"✅ Conversation complete! Total turns: {len(questions)}")
print(f"📝 Total messages in context: {len(conversation)}")

---
## 🧠 Example 7: Complex Reasoning & Long-Context Analysis

Sonnet 4.6 approaches Opus-level intelligence for complex reasoning. It excels at multi-source analysis across legal, financial, and technical content — all within its 1M token context window.

In [ ]:
# @title Complex Multi-Step Reasoning
reasoning_prompt = """
You are a senior data scientist. Analyze this scenario and provide a complete, structured recommendation:

## Scenario
An e-commerce company has the following data:
- Monthly active users: 2.4M
- Conversion rate: 2.1% (industry avg: 3.2%)
- Average order value: $67
- Cart abandonment rate: 78%
- Mobile traffic: 65% (but mobile conversion: only 1.1%)
- Email open rate: 18%, click rate: 2.3%
- Customer return rate: 22% within 60 days
- Top traffic sources: Organic (35%), Paid (28%), Social (22%), Direct (15%)

## Task
1. Identify the TOP 3 problems from this data
2. For each problem, propose a data-driven solution with expected impact
3. Prioritize solutions by ROI (calculate estimated revenue impact)
4. Propose an A/B testing plan for the highest-priority solution
5. Identify what additional data you would need to refine these recommendations
"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a senior data scientist and growth strategist. Be quantitative and specific with your analysis."},
        {"role": "user", "content": reasoning_prompt}
    ],
    max_tokens=4096
)

print(response.choices[0].message.content)

---
## 🤖 Example 8: Agentic Workflow — Research & Report Agent

Sonnet 4.6 excels at agentic workflows with independent operation, sub-task identification, and self-correction. Let's build a mini research agent that plans, gathers data, and synthesizes a report.

In [ ]:
# @title Agentic Research Workflow
import json

# Define agent tools
agent_tools = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "Search the web for information on a topic",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search query"}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_data",
            "description": "Analyze data and extract key insights",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {"type": "string", "description": "Data to analyze"},
                    "analysis_type": {"type": "string", "enum": ["summary", "trends", "comparison", "forecast"]}
                },
                "required": ["data", "analysis_type"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "write_report",
            "description": "Write a structured report section",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {"type": "string", "description": "Section title"},
                    "content": {"type": "string", "description": "Section content"},
                    "section_type": {"type": "string", "enum": ["introduction", "findings", "analysis", "conclusion"]}
                },
                "required": ["title", "content", "section_type"]
            }
        }
    }
]

# Simulate tool responses
def execute_agent_tool(name, args):
    if name == "search_web":
        query = args.get('query', '')
        return json.dumps({
            "results": [
                {"title": f"Recent developments in {query}", "snippet": f"Latest research shows significant advances in {query} with new breakthroughs in efficiency and performance."},
                {"title": f"{query} market analysis 2026", "snippet": f"The {query} market is projected to grow 35% YoY, driven by enterprise adoption and new use cases."},
                {"title": f"Expert opinions on {query}", "snippet": f"Industry experts believe {query} will transform how businesses operate within the next 2-3 years."}
            ]
        })
    elif name == "analyze_data":
        return json.dumps({"insights": ["Strong growth trajectory", "Increasing enterprise adoption", "Cost efficiency improvements"], "confidence": 0.87})
    elif name == "write_report":
        return json.dumps({"status": "success", "section": args.get('section_type'), "word_count": len(args.get('content', '').split())})
    return json.dumps({"error": "Unknown tool"})

# Run the agent
agent_messages = [
    {"role": "system", "content": """You are a research agent. Your task is to research a topic using available tools and produce a comprehensive report.
Follow this workflow:
1. PLAN: Outline your research approach
2. GATHER: Use search_web to find information
3. ANALYZE: Use analyze_data to extract insights
4. REPORT: Use write_report to compile findings

Think step by step and use tools strategically."""},
    {"role": "user", "content": "Research the current state of AI agents in 2026 and create a brief report on the key trends."}
]

print("🤖 Agent Starting Research...")
print("=" * 60)

# Agent loop (max 3 iterations)
for step in range(3):
    response = client.chat.completions.create(
        model=MODEL,
        messages=agent_messages,
        tools=agent_tools,
        max_tokens=2048
    )

    message = response.choices[0].message

    if message.content:
        print(f"\n💭 Agent Thinking (Step {step+1}):\n{message.content[:300]}..." if len(message.content or '') > 300 else f"\n💭 Agent (Step {step+1}):\n{message.content}")

    if not message.tool_calls:
        print(f"\n{'='*60}")
        print("✅ Agent completed! Final response:")
        print(message.content)
        break

    # Process tool calls
    agent_messages.append({"role": "assistant", "content": message.content, "tool_calls": [{"id": tc.id, "type": "function", "function": {"name": tc.function.name, "arguments": tc.function.arguments}} for tc in message.tool_calls]})

    for tc in message.tool_calls:
        args = json.loads(tc.function.arguments)
        result = execute_agent_tool(tc.function.name, args)
        print(f"   🔧 Tool: {tc.function.name}({json.dumps(args)[:80]}...)")
        agent_messages.append({"role": "tool", "tool_call_id": tc.id, "content": result})

print(f"\n📊 Total agent steps: {step + 1}")

---
## 📈 Token Usage & Cost Tracking

In [17]:
# @title Check Token Usage (Last Response)
if hasattr(response, 'usage') and response.usage:
    usage = response.usage
    input_cost = (usage.prompt_tokens / 1_000_000) * 3.0
    output_cost = (usage.completion_tokens / 1_000_000) * 15.0
    total_cost = input_cost + output_cost

    print("📊 Token Usage (Last Request):")
    print(f"   • Prompt tokens:     {usage.prompt_tokens:,}")
    print(f"   • Completion tokens: {usage.completion_tokens:,}")
    print(f"   • Total tokens:      {usage.total_tokens:,}")
    print(f"\n💰 Estimated Cost:")
    print(f"   • Input:  ${input_cost:.6f}")
    print(f"   • Output: ${output_cost:.6f}")
    print(f"   • Total:  ${total_cost:.6f}")
    print(f"\n💡 Claude Sonnet 4.6 Pricing (via OpenRouter):")
    print(f"   • Input: $3 / 1M tokens")
    print(f"   • Output: $15 / 1M tokens")
else:
    print("Token usage not available in response")
    print("\n💡 Claude Sonnet 4.6 Pricing (via OpenRouter):")
    print("   • Input: $3 / 1M tokens")
    print("   • Output: $15 / 1M tokens")

📊 Token Usage (Last Request):
   • Prompt tokens:     2,488
   • Completion tokens: 1,163
   • Total tokens:      3,651

💰 Estimated Cost:
   • Input:  $0.007464
   • Output: $0.017445
   • Total:  $0.024909

💡 Claude Sonnet 4.6 Pricing (via OpenRouter):
   • Input: $3 / 1M tokens
   • Output: $15 / 1M tokens


---
## 🎯 Key Takeaways

**Claude Sonnet 4.6 excels at:**
- ✅ **Coding** — 70% preferred over Sonnet 4.5; less overengineering, better context awareness
- ✅ **Tool Calling** — Parallel execution, complex multi-tool workflows
- ✅ **Instruction Following** — Fewer hallucinations, better follow-through
- ✅ **Long Context** — 1M token window with effective reasoning across all of it
- ✅ **Agent Planning** — Long-horizon strategy with self-correction
- ✅ **Structured Output** — Reliable JSON mode for data pipelines
- ✅ **Computer Use** — Major OSWorld benchmark improvements
- ✅ **Design** — Polished frontend code, layouts, and animations

**Best for:**
- 💻 Software development & code review
- 🤖 Building agentic AI workflows
- 📊 Data analysis & reporting
- 🏢 Enterprise knowledge work
- 🎨 Frontend design & prototyping

**Pricing advantage:**
- Opus-level performance at Sonnet pricing ($3/$15 per M tokens)

📖 *Learn more: [Anthropic Blog](https://www.anthropic.com/news/claude-sonnet-4-6) | [OpenRouter](https://openrouter.ai/anthropic/claude-sonnet-4.6)*

---
*Notebook by [@BuildFastWithAI](https://www.buildfastwithai.com)*